# Prototipo para proceso ETL de la base de datos


## 1. Cargue de librerias y base de datos

In [1]:
## Cargue de librerias
import pandas as pd
import numpy as np
import re

In [2]:
## Cargue de la base de datos
ruta = input("Ingresar la ruta del archivo House_Rent_Dataset.csv (sin comillas): ").strip()
original_df_0 = pd.read_csv(ruta, sep = ",")
original_df_0.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


## 2. Ajuste del formato de los datos
- Formato de las variables : corroborar que sean coherentes y transformar por si acaso
- Variable Floor: Revisar si es mejor dividirla en dos: piso de residencia // cant de pisos del edificio
- Dicotomizando variables categoricas (trato especial a Tenant Preferred, esta es dicotoma sin omitir registro para categorias compartidas)
- Seleccion final de variables : solo las que se emplearan

Estos son los pasos que se deberán plasmar en el archivo data_loader.  

Que genere el cargue de datos y tambien realice la depuración de los mismos para ya luego entrenar / usar el modelo propuesto

In [4]:
## Revisiones de formato de variables
original_df_0.info() # De paso no hay nulos 😎
original_df = original_df_0.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Posted On          4746 non-null   object
 1   BHK                4746 non-null   int64 
 2   Rent               4746 non-null   int64 
 3   Size               4746 non-null   int64 
 4   Floor              4746 non-null   object
 5   Area Type          4746 non-null   object
 6   Area Locality      4746 non-null   object
 7   City               4746 non-null   object
 8   Furnishing Status  4746 non-null   object
 9   Tenant Preferred   4746 non-null   object
 10  Bathroom           4746 non-null   int64 
 11  Point of Contact   4746 non-null   object
dtypes: int64(4), object(8)
memory usage: 445.1+ KB


In [5]:
# # Ajuste variable floor
original_df['residence_floor'] = original_df['Floor'].str.split(' out of ', expand=True)[0]
original_df['residence_max_floor'] = original_df['Floor'].str.split(' out of ', expand=True)[1]

# Con nombre de piso
original_df['residence_floor'] = original_df['residence_floor'].apply(lambda x: re.sub(r'Ground', '1', x))         # Ground se vuelve el piso 1
original_df['residence_floor'] = original_df['residence_floor'].apply(lambda x: re.sub(r'Lower Basement', '1', x)) # Lower Basement se vuelve el piso 1
original_df['residence_floor'] = original_df.apply(lambda row: re.sub(r'Upper Basement', row['residence_max_floor'], row['residence_floor']) if row['residence_floor'] == 'Upper Basement' else row['residence_floor'], axis=1)
original_df['residence_max_floor'] = original_df.apply(lambda row: row['residence_floor'] if pd.isnull(row['residence_max_floor']) else row['residence_max_floor'], axis=1)

# Convertir a numericas
original_df['residence_floor'] = original_df['residence_floor'].astype(int)
original_df['residence_max_floor'] = original_df['residence_max_floor'].astype(int)

In [6]:
# Revision de inconsistencias
original_df[original_df['residence_floor'] > original_df['residence_max_floor']]

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact,residence_floor,residence_max_floor
105,2022-06-06,1,6000,600,8 out of 5,Carpet Area,Pratiraksha Nagar,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner,8,5
161,2022-06-27,2,10000,450,2 out of 1,Carpet Area,Behala,Kolkata,Semi-Furnished,Bachelors/Family,2,Contact Owner,2,1


In [7]:
# # Revision variable Tenant Preferred
original_df["Tenant_bachelors"] = original_df["Tenant Preferred"].apply(lambda x: 1 if "Bachelors" in x else 0)
original_df["Tenant_family"] = original_df["Tenant Preferred"].apply(lambda x: 1 if "Family" in x else 0)

In [8]:
# # Selección de variables
original_df = original_df[['Rent',	'BHK',	'Size',	'Area Type',	'City',	'Furnishing Status',	'Bathroom',	'Point of Contact',	'residence_floor',	'residence_max_floor',	'Tenant_bachelors',	'Tenant_family']]

In [9]:
# original_df[:5]
original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Rent                 4746 non-null   int64 
 1   BHK                  4746 non-null   int64 
 2   Size                 4746 non-null   int64 
 3   Area Type            4746 non-null   object
 4   City                 4746 non-null   object
 5   Furnishing Status    4746 non-null   object
 6   Bathroom             4746 non-null   int64 
 7   Point of Contact     4746 non-null   object
 8   residence_floor      4746 non-null   int32 
 9   residence_max_floor  4746 non-null   int32 
 10  Tenant_bachelors     4746 non-null   int64 
 11  Tenant_family        4746 non-null   int64 
dtypes: int32(2), int64(6), object(4)
memory usage: 408.0+ KB


In [10]:
# Dicotomizando caracteristicas tipo object
original_df = pd.get_dummies(original_df, columns=['Area Type', 'City', 'Furnishing Status', 'Point of Contact'], drop_first=True).astype(int)
original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 19 columns):
 #   Column                            Non-Null Count  Dtype
---  ------                            --------------  -----
 0   Rent                              4746 non-null   int32
 1   BHK                               4746 non-null   int32
 2   Size                              4746 non-null   int32
 3   Bathroom                          4746 non-null   int32
 4   residence_floor                   4746 non-null   int32
 5   residence_max_floor               4746 non-null   int32
 6   Tenant_bachelors                  4746 non-null   int32
 7   Tenant_family                     4746 non-null   int32
 8   Area Type_Carpet Area             4746 non-null   int32
 9   Area Type_Super Area              4746 non-null   int32
 10  City_Chennai                      4746 non-null   int32
 11  City_Delhi                        4746 non-null   int32
 12  City_Hyderabad                    

In [11]:
original_df[:5]

,Rent,BHK,Size,Bathroom,residence_floor,residence_max_floor,Tenant_bachelors,Tenant_family,Area Type_Carpet Area,Area Type_Super Area,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Point of Contact_Contact Builder,Point of Contact_Contact Owner
0,10000,2,1100,2,1,2,1,1,0,1,0,0,0,1,0,0,1,0,1
1,20000,2,800,1,1,3,1,1,0,1,0,0,0,1,0,1,0,0,1
2,17000,2,1000,1,1,3,1,1,0,1,0,0,0,1,0,1,0,0,1
3,10000,2,800,1,1,2,1,1,0,1,0,0,0,1,0,0,1,0,1
4,7500,2,850,1,1,2,1,0,1,0,0,0,0,1,0,0,1,0,1


In [12]:
original_df.to_csv('House_Rent_Dataset_cleaned.csv', index=False)